# ML Pipeline Preparation

### 1. Importing libraries and loading data from data base

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime
import collections
import copy


from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.multioutput import MultiOutputClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import classification_report

### Optional display settings

In [ ]:
plt.style.use('seaborn-dark')
pd.set_option("display.max_rows", 1000)

In [ ]:
engine = create_engine('sqlite:///Pipeline_Project.db')
df = pd.read_sql_table('Messages', engine)
df.info()

### General division into labels and features

In [ ]:
X = df.iloc[:, 1]
Y = df.iloc[:, 2:]

In [ ]:
### Weights adjustments

In [ ]:
#class_weights = []

In [ ]:
"""for feature in Y:
    dictionary = {(Y[feature].value_counts()/len(Y)).index[0]: (Y[feature].value_counts()/len(Y)).values[1],
                  (Y[feature].value_counts()/len(Y)).index[1]: (Y[feature].value_counts()/len(Y)).values[0]}

    class_weights.append(dictionary)
"""    

In [ ]:
"""for dictionary in copy.copy(class_weights):
    dictionary = {k: dictionary[k] for k in sorted(dictionary)}
    
    class_weights.pop(0)
    class_weights.append(dictionary)
"""

In [ ]:
"""
for element in class_weights:
    for key, value in element.items():
        print(type(key), type(value))
        
"""

In [ ]:
#compute_sample_weight(class_weights, Y)

### 2. Key functions

#### Tokenizer for processing the text data

In [ ]:
def tokenize(text):
    #Normalization - lowercase  - no punctuation removal - the nltk.tokenize should interpret them by itself
    text = text.lower()
    ######text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    #Toeknization
    ######words = text.split()
    words = word_tokenize(text)
    #Stop words removal
    words = [w for w in words if w not in stopwords.words("english")]
    
    #Named Entities??
    # Perhaps
    
    #Lemmatization
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    
    return lemmed

#### Since I want to test out many models, it seems reasonable to write a function which will make pipeline with keyword arguments

In [ ]:
def make_sklearn_pipeline(Pipeline = Pipeline, memory = None, verbose = False, **kwargs):
    """
    Firt argument is sklearn Pipepline class. It should not be changed.
    
    Definition of pipeline steps happens EXPLICITLY within the instantiation!
    Definition of typical pipeline looks like following:
    example_pipeline = make_sklearn_pipeline(steps = [('name#1',function#1),
                                                      ('name#2',function#2),
                                                      ('name#3',function#3),
                                                      ('name#4',function#4)....], verbose = ...,
                                                      memory = ...)
    
    It is not required to give values for ´verbose´ and ´memory´. They have default values as False
    and None respectively. For more information visit: 
    ´https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html´
    """
    pipeline = Pipeline(steps = kwargs['steps'], memory = memory, verbose = verbose)
    
    return pipeline

#### Function transforming the classification report into data frame with recall, precision and accuracy only 

In [ ]:
def df_from_sklearn_cl_reports(cl_reports):

    data_frame = pd.DataFrame()

    for feature in list(cl_reports.keys()):
        l = cl_reports[feature].split(' ')
        l = [x for x in l if x !='']
        l = [x for x in l if '\n' not in x]
        l = l[:l.index('accuracy')]
        columns = l[:3]
        l = [x for x in l if x not in columns]
        labels = []
        precisions = []
        recalls = []
        f1_scores = []
    
        for i, element in enumerate(l):
            if i == 0 or i % 4 == 0:
                labels.append(element)
                precisions.append(l[i+1])
                recalls.append(l[i+2])
                f1_scores.append(l[i+3])
            
        comunicates = [feature]*len(labels)
    
        if data_frame.shape[0] == 0:      
            data_frame['communicate'] = comunicates
            data_frame['label'] = labels
            data_frame['precisions'] = precisions
            data_frame['recalls'] = recalls
            data_frame['f1_scores'] = f1_scores
        
        else:
            auxiliary_df = pd.DataFrame()
            auxiliary_df['communicate'] = comunicates
            auxiliary_df['label'] = labels
            auxiliary_df['precisions'] = precisions
            auxiliary_df['recalls'] = recalls
            auxiliary_df['f1_scores'] = f1_scores 
        
            data_frame = pd.concat([data_frame, auxiliary_df])
        
            del auxiliary_df
    
    data_frame.set_index(["communicate"], inplace = True)
    
    #Chaning the data types
    data_frame.label = data_frame.label.astype("float")
    data_frame.label = data_frame.label.astype("int")
    data_frame.precisions = data_frame.precisions.astype("float")
    data_frame.recalls = data_frame.recalls.astype("float")
    data_frame.f1_scores = data_frame.f1_scores.astype("float")
    
    return data_frame    

### 3. Building a machine learning pipeline

#### 3.1 AdaBoost Classifier

##### Definition and fitting

In [ ]:
adaboost_with_transformer = make_sklearn_pipeline(verbose = True, steps = [('vect', CountVectorizer(tokenizer = tokenize)),
                           ('tfidf', TfidfTransformer()),                                              
                           ('clf', MultiOutputClassifier(estimator = AdaBoostClassifier(
                           DecisionTreeClassifier(class_weight = "balanced"))))])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)


In [ ]:
adaboost_with_transformer.fit(X_train, y_train)

##### Predicting and classification report

In [ ]:
y_pred_ada = adaboost_with_transformer.predict(X_test)
y_pred_ada = pd.DataFrame(y_pred_ada, columns = list(Y.columns))
y_test = y_test.reset_index(drop = True)
for i, var in enumerate(Y):
    print(var)
    print(classification_report(y_test.iloc[:,i], y_pred_ada.iloc[:,i]))

##### Providing a data frame for easier results assessment

In [ ]:
classification_reports = {}

In [ ]:
for i, var in enumerate(Y):
    classification_reports[var] = (classification_report(y_test.iloc[:,i], y_pred_ada.iloc[:,i]));

In [ ]:
adaboost_with_transformer_report_df = df_from_sklearn_cl_reports(classification_reports)

In [ ]:
adaboost_with_transformer_report_df.loc[
    adaboost_with_transformer_report_df.label == 1].groupby(
    ["communicate"])["precisions"].mean().sort_values(ascending = False).mean()

In [ ]:
adaboost_with_transformer_report_df.loc[
    adaboost_with_transformer_report_df.label == 1].groupby(
    ["communicate"])["recalls"].mean().sort_values(ascending = False).mean()

In [ ]:
RandomForest_with_transformer = make_sklearn_pipeline(verbose = True, steps = [('vect', CountVectorizer(tokenizer = tokenize)),
                           ('tfidf', TfidfTransformer()),                                              
                           ('clf', MultiOutputClassifier(estimator = RandomForestClassifier
                                                         (class_weight = "balanced")))])

In [ ]:
RandomForest_with_transformer.fit(X_train, y_train)

In [ ]:
y_pred_forest = RandomForest_with_transformer.predict(X_test)
y_pred_forest = pd.DataFrame(y_pred_forest, columns = list(Y.columns))
y_test = y_test.reset_index(drop = True)
for i, var in enumerate(Y):
    print(var)
    print(classification_report(y_test.iloc[:,i], y_pred_forest.iloc[:,i]))

In [ ]:
classification_reports.clear()

In [ ]:
for i, var in enumerate(Y):
    classification_reports[var] = (classification_report(y_test.iloc[:,i], y_pred_forest.iloc[:,i]));

In [ ]:
forest_with_transformer_report_df = df_from_sklearn_cl_reports(classification_reports)

In [ ]:
forest_with_transformer_report_df.loc[
    forest_with_transformer_report_df.label == 1].groupby(
    ["communicate"])["precisions"].mean().sort_values(ascending = False).mean()

In [ ]:
forest_with_transformer_report_df.loc[
    forest_with_transformer_report_df.label == 1].groupby(
    ["communicate"])["recalls"].mean().sort_values(ascending = False).mean()

##### Trying to ehence the model's performance via GridSearch

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [100,1000]    
}

In [ ]:
RandomForest_optimized = GridSearchCV(RandomForest_with_transformer, param_grid = parameters)

In [ ]:
#Attention! This cell may take a bit time to perform (around 15 minutes on typical PC)
RandomForest_optimized.fit(X_train, y_train)

In [ ]:
y_pred_opt = RandomForest_optimized.predict(X_test)
y_pred_opt = pd.DataFrame(y_pred_opt, columns = list(Y.columns))
y_test = y_test.reset_index(drop = True)
for i, var in enumerate(Y):
    print(var)
    print(classification_report(y_test.iloc[:,i], y_pred_opt.iloc[:,i]))

In [ ]:
classification_reports.clear()

In [ ]:
for i, var in enumerate(Y):
    classification_reports[var] = (classification_report(y_test.iloc[:,i], y_pred_opt.iloc[:,i]));

In [ ]:
RandomForest_optimized_report_df = df_from_sklearn_cl_reports(classification_reports)

In [ ]:
RandomForest_optimized_report_df.loc[
    RandomForest_optimized_report_df.label == 1].groupby(
    ["communicate"])["precisions"].mean().sort_values(ascending = False).mean()

In [ ]:
RandomForest_optimized_report_df.loc[
    RandomForest_optimized_report_df.label == 1].groupby(
    ["communicate"])["recalls"].mean().sort_values(ascending = False).mean()

In [ ]:
classification_reports.clear()

In [ ]:
dt_depth_2 = DecisionTreeClassifier(max_depth=2, min_samples_leaf=1, class_weight = "balanced")
dt_depth_9 = DecisionTreeClassifier(max_depth=9, min_samples_leaf=1, class_weight = "balanced")

In [ ]:
parameters = {
    'clf__estimator__base_estimator': [dt_depth_2,dt_depth_9],
    'clf__estimator__n_estimators': [50,100]    
}

In [ ]:
adaboost_transf_optimized = GridSearchCV(adaboost_with_transformer, param_grid = parameters)

In [ ]:
#Attention! This cell may take a bit time to perform (around 15 minutes on typical PC)
adaboost_transf_optimized.fit(X_train, y_train)

In [ ]:
y_pred_opt_ada = adaboost_transf_optimized.predict(X_test)
y_pred_opt_ada = pd.DataFrame(y_pred_opt_ada, columns = list(Y.columns))
y_test = y_test.reset_index(drop = True)
for i, var in enumerate(Y):
    print(var)
    print(classification_report(y_test.iloc[:,i], y_pred_opt_ada.iloc[:,i]))

In [ ]:
for i, var in enumerate(Y):
    classification_reports[var] = (classification_report(y_test.iloc[:,i], y_pred_opt_ada.iloc[:,i]));

In [ ]:
adaboost_optimized_report_df = df_from_sklearn_cl_reports(classification_reports)

In [ ]:
adaboost_optimized_report_df.loc[
    adaboost_optimized_report_df.label == 1].groupby(
    ["communicate"])["precisions"].mean().sort_values(ascending = False).mean()

In [ ]:
adaboost_optimized_report_df.loc[
    adaboost_optimized_report_df.label == 1].groupby(
    ["communicate"])["recalls"].mean().sort_values(ascending = False).mean()

>Undersampling still gives poor results - escpecially for recall which is very important in disaster response system. Undersampling may not necesserilly work good in multioutput problems, because of possible noise beeing added to the data set.
Moreover, if we really want to balance all of the classes via undersampling we will end up with very small data set (some imbalanced classes are much less numerous than the better balanced ones). Hence the next iterration will be conducted with **´class weights´** adjustments and without any reduction of the data set size.

In [ ]:
adaboost_optimized_report_df.to_csv("Adaboost_DT_GridSearch_depth2and9_estimators_50and100.csv", 
                                    index = True)